In [1]:
import numpy as np
import math
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [2]:
#planetary constants
G           = 6.67e-11
Mb          = 4.0e30                    # black hole
Ms          = 2.0e30                    # sun
Me          = 5.972e24                  # earth        
Mm          = 6.39e23                   # mars
Mc          = 6.39e20                   # unknown comet
au          = 1.5e11
daysec      = 24.0*60*60

In [3]:
data = 'PS_2023.01.27_19.53.08.csv'

exoplanets = pd.read_csv(data)

attr = ['pl_name','hostname', 'sy_pnum','pl_orbsmax', 'pl_orbper','pl_orbeccen', 'pl_rade', 'pl_bmasse', 'discoverymethod','st_mass', 'st_rad' ]
unary_systems = exoplanets[exoplanets['sy_snum'] == 1]
usable_unaries = unary_systems[np.logical_and(np.logical_and(unary_systems['pl_orbeccen'] > 0, ~unary_systems['pl_rade'].isna()), ~unary_systems['pl_bmasse'].isna())][attr]
usable_unaries[attr].head()
kep_51 = usable_unaries[usable_unaries['hostname'] == 'Kepler-51'].groupby('pl_name', as_index= False).max()
kep_51.head()

,pl_name,hostname,sy_pnum,pl_orbsmax,pl_orbper,pl_orbeccen,pl_rade,pl_bmasse,discoverymethod,st_mass,st_rad
0,Kepler-51 b,Kepler-51,3,0.2514,45.1542,0.040,7.1,3.69,Transit,1.04,0.94
1,Kepler-51 c,Kepler-51,3,0.3840,85.3139,0.014,9.0,4.43,Transit,1.04,0.94
2,Kepler-51 d,Kepler-51,3,0.5090,130.1940,0.010,9.7,7.60,Transit,1.04,0.94


In [4]:
def spheres(size, clr, dist=0): 
    
    # Set up 100 points. First, do angles
    theta = np.linspace(0,2*np.pi,100)
    phi = np.linspace(0,np.pi,100)
    
    # Set up coordinates for points on the sphere
    x0 = dist + size * np.outer(np.cos(theta),np.sin(phi))
    y0 = size * np.outer(np.sin(theta),np.sin(phi))
    z0 = size * np.outer(np.ones(100),np.cos(phi))
    
    # Set up trace
    trace= go.Surface(x=x0, y=y0, z=z0, colorscale=[[0,clr], [1,clr]])
    trace.update(showscale=False)

    return trace

In [5]:
def orbits(dist, offset=0, clr='white', wdth=2): 
    
    # Initialize empty lists for eac set of coordinates
    xcrd=[]
    ycrd=[]
    zcrd=[]
    
    # Calculate coordinates
    for i in range(0,361):
        xcrd=xcrd+[(round(np.cos(math.radians(i)),5)) * dist + offset]
        ycrd=ycrd+[(round(np.sin(math.radians(i)),5)) * dist]
        zcrd=zcrd+[0]
    
    trace = go.Scatter3d(x=xcrd, y=ycrd, z=zcrd, marker=dict(size=0.1), line=dict(color=clr,width=wdth))
    return trace

In [6]:
def annot(xcrd, zcrd, txt, xancr='center'):
    strng=dict(showarrow=False, x=xcrd, 
               y=0, z=zcrd, text=txt, 
               xanchor=xancr, font=dict(color='white',size=12))
    return strng

In [7]:
float(kep_51[kep_51['pl_name'] == 'Kepler-51 b']['pl_orbsmax']*149.6)

37.60944

In [9]:
float(kep_51['st_mass'].max()*20000)

20800.0

In [10]:
float(kep_51[kep_51['pl_name'] == 'Kepler-51 b']['pl_rade']*12756)

90567.59999999999

In [15]:
diameter = [float(kep_51['st_mass'].max()*109), float(kep_51[kep_51['pl_name'] == 'Kepler-51 b']['pl_rade']), float(kep_51[kep_51['pl_name'] == 'Kepler-51 c']['pl_rade']), float(kep_51[kep_51['pl_name'] == 'Kepler-51 d']['pl_rade'])]
#diameter = [((i / 12756) * 2) for i in diameter_km]
distance_from_sun = [0, float(kep_51[kep_51['pl_name'] == 'Kepler-51 b']['pl_orbsmax']*1496), float(kep_51[kep_51['pl_name'] == 'Kepler-51 c']['pl_orbsmax']*1496), float(kep_51[kep_51['pl_name'] == 'Kepler-51 d']['pl_orbsmax']*1496)]

# Create spheres for the Sun and planets
trace0=spheres(diameter[0], '#ffff00', distance_from_sun[0]) # Kepler-51
trace1=spheres(diameter[1], '#87877d', distance_from_sun[1]) # Kepler-51 b
trace2=spheres(diameter[2], '#d23100', distance_from_sun[2]) # Kepler-51 c
trace3=spheres(diameter[3], '#325bff', distance_from_sun[3]) # Kepler-51 d

# Set up orbit traces
trace11 = orbits(distance_from_sun[1]) # Kepler-51 b
trace12 = orbits(distance_from_sun[2]) # Kepler-51 c
trace13 = orbits(distance_from_sun[3]) # Kepler-51 d

star = 'Kepler-51'
axis_title = f'Distance from {star}'
title = f'{star} star system'


layout=go.Layout(title = title, showlegend=False, margin=dict(l=0, r=0, t=0, b=0),
                    paper_bgcolor = 'black',
                    scene = dict(xaxis=dict(title=axis_title,
                                        titlefont_color='black', 
                                        range=[-4000,4000], 
                                        backgroundcolor='black',
                                        color='black',
                                        gridcolor='black'),
                                yaxis=dict(title='Distance from the Sun',
                                        titlefont_color='black',
                                        range=[-4000,4000],
                                        backgroundcolor='black',
                                        color='black',
                                        gridcolor='black'
                                        ),
                                zaxis=dict(title='', 
                                        range=[-4000,4000],
                                        backgroundcolor='black',
                                        color='white', 
                                        gridcolor='black'
                                        ),
                                annotations=[
                                   annot(distance_from_sun[0], 40, 'Kepler-51', xancr='left'),
                                   annot(distance_from_sun[1], 5, 'Kepler-51 b'),
                                   annot(distance_from_sun[2], 9, 'Kepler-51 c'),
                                   annot(distance_from_sun[3], 9, 'Kepler-51 d'),
                                   ]
                            ))

fig = go.Figure(data = [trace0, trace1, trace2, trace3,
                        trace11, trace12, trace13],layout = layout)

fig.show()